In [267]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
import numpy as np
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from mtcnn import MTCNN
import os

import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# 1. Modify ResNet-50 to output embeddings


In [181]:
# Load pre-trained ResNet-50
model = models.resnet50(pretrained=True)

# Modify the final layer to output embeddings (e.g., 128-dimensional embedding)
model.fc = nn.Linear(model.fc.in_features, 128)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 2. Prepare Dataset and DataLoader


In [182]:
# Define transformations

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }

In [183]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.utils.data import DataLoader, Subset, Dataset

# Load dataset using ImageFolder
data_dir = './celeb_dataset/img_align_celeba/'

full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Split into train and validation sets
total_size = len(full_dataset)
indices = list(range(total_size))
random.shuffle(indices)
train_size = int(0.8 * total_size)
train_indices = indices[:train_size]
val_indices = indices[train_size:]

# Create subsets for training and validation
train_subset = Subset(full_dataset, train_indices)
val_subset = Subset(full_dataset, val_indices)

# 3. Define Triplet Loss Function


In [184]:
import torch.nn.functional as F

# Define Triplet Loss function
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        distance_positive = F.pairwise_distance(anchor, positive)
        distance_negative = F.pairwise_distance(anchor, negative)
        losses = F.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()


# 4. Create a Custom Dataset for Triplets

In [185]:
import torch
from torch.utils.data import Dataset
import random

class TripletImageDataset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices
        self.transform=transform
        self.targets = [dataset.targets[i] for i in indices]

    def __getitem__(self, index):
        anchor_idx = self.indices[index]
        anchor_img, anchor_label = self.dataset[anchor_idx]

        # Positive
        positive_indices = [i for i, label in enumerate(self.targets) if label == anchor_label and i != anchor_idx]
        positive_idx = random.choice(positive_indices) if positive_indices else anchor_idx
        positive_img, _ = self.dataset[positive_idx]

        # Negative
        negative_indices = [i for i, label in enumerate(self.targets) if label != anchor_label]
        negative_idx = random.choice(negative_indices) if negative_indices else anchor_idx
        negative_img, _ = self.dataset[negative_idx]

        return anchor_img, positive_img, negative_img

    def __len__(self):
        return len(self.indices)

# 5. Training Loop with Triplet Loss

In [186]:
# Create DataLoaders
train_dataset = TripletImageDataset(full_dataset, train_indices)
val_dataset = TripletImageDataset(full_dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# Verify the sizes
print(f"Training set size: {len(train_loader.dataset)}")
print(f"Validation set size: {len(val_loader.dataset)}")

Training set size: 162079
Validation set size: 40520


In [187]:
print(type(transforms))
print(type(full_dataset))


<class 'module'>
<class 'torchvision.datasets.folder.ImageFolder'>


In [188]:
train_loader

In [189]:
class TripletImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        #print(idx)
        anchor_path = self.image_paths[idx]
        anchor_image = Image.open(anchor_path)

        positive_path = anchor_path  # Same as anchor for simplicity
        positive_image = Image.open(positive_path)

        negative_path = random.choice(self.image_paths)
        negative_image = Image.open(negative_path)

        if self.transform:
            anchor_image = self.transform(anchor_image)
            positive_image = self.transform(positive_image)
            negative_image = self.transform(negative_image)

        return anchor_image, positive_image, negative_image


In [190]:
image_paths = [ os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

In [191]:
# ac=image_paths[39021]
# print(ac)
# # anchor_image = Image.open(ac).convert('RGB')
# # # Load a sample image
# # sample_image_path = 'path/to/sample_image.jpg'
# sample_image = Image.open(ac).convert('RGB')
# print(sample_image.size) 
# # Apply transformations
# transformed_image = data_transforms(sample_image)
# print(transformed_image.size)

In [192]:
from torch.utils.data import DataLoader

data_dir = './celeb_dataset/img_align_celeba/img_align_celeba'

# Create dataset and dataloader
dataset = TripletImageDataset(folder_path=data_dir, transform=data_transforms)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)


In [193]:
# Test DataLoader
for anchor, positive, negative in train_loader:
    print(anchor.shape, positive.shape, negative.shape)
    break


torch.Size([16, 3, 224, 224]) torch.Size([16, 3, 224, 224]) torch.Size([16, 3, 224, 224])


In [251]:
import torch
import torch.nn as nn
from torchvision import models

class TripletResNet50(nn.Module):
    def __init__(self, embedding_dim=128):
        super(TripletResNet50, self).__init__()
        # Load a pre-trained ResNet model
        self.resnet = models.resnet50(pretrained=True)
        # Remove the final fully connected layer
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])  # Excludes the last layer
        # Define the embedding layer
        # Calculate the correct input features size for the new linear layer
        self.flatten = nn.Flatten()
        self.embedding_dim = 128  # The desired embedding dimension
        self.fc_input_dim = 2048   
        
        self.embedding = nn.Linear(self.fc_input_dim*7*7, embedding_dim)

    def forward(self, x):
        #print(self.flatten)
        features = self.resnet(x)  # This outputs features before the final FC layer
        features = self.flatten(features)  # Flatten the output to match input for linear layer
        embeddings = self.embedding(features)
        print("Features shape:", features.shape)  # Should match input to self.embedding
        #embeddings = self.embedding(features)
        print("Embeddings shape:", embeddings.shape)
        return embeddings


# Example usage
model = TripletResNet(embedding_dim=128)

In [252]:
import torch.optim as optim

def train_triplet(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for anchor, positive, negative in dataloader:
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            anchor_output = model(anchor)
            positive_output = model(positive)
            negative_output = model(negative)

            # Compute triplet loss
            loss = criterion(anchor_output, positive_output, negative_output)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * anchor.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')


In [253]:
# Ensure device is set to CPU
device = torch.device('cpu')

# Initialize model, criterion, and optimizer
model = TripletResNet50(embedding_dim=128).to(device)
criterion = nn.TripletMarginLoss(margin=1.0).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [254]:
train_triplet(model, train_loader, criterion, optimizer, num_epochs=10)

# Save the model
torch.save(model.state_dict(), 'triplet_resnet50_cpu.pth')

Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([16, 100352])
Embeddings shape: torch.Size([16, 128])
Features shape: torch.Size([

KeyboardInterrupt: 

In [255]:
# Save the model
torch.save(model.state_dict(), 'triplet_resnet50_cpu.pth')

## Extract Embeddings  

In [256]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
import numpy as np
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from mtcnn import MTCNN
import os


In [262]:
# Initialize the face detector (MTCNN) and feature extractor (ResNet50)
detector = MTCNN()

# Initialize the model
model = TripletResNet50(embedding_dim=128)

# Load the model's state dictionary
model.load_state_dict(torch.load('./triplet_resnet50_cpu.pth'))

# Set the model to evaluation mode
model.eval()


TripletResNet50(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv

In [263]:
# Preprocessing pipeline for images
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [264]:
# Path to your small dataset of celebrity images
celeba_img_path = './celeb_dataset/img_align_celeba/img_align_celeba/'  # Replace with your small dataset directory

# Load image file paths from the small dataset
celeba_image_files = os.listdir(celeba_img_path)
celeba_image_paths = [os.path.join(celeba_img_path, img_file) for img_file in celeba_image_files]



In [265]:
# Function to extract face embeddings from an image
def extract_face_embedding(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face in the image
    detection = detector.detect_faces(image_rgb)

    if len(detection) > 0:
        x, y, w, h = detection[0]['box']
        face = image_rgb[y:y+h, x:x+w]
        face = Image.fromarray(face)
        face_tensor = preprocess(face).unsqueeze(0)

        with torch.no_grad():
            embedding = model(face_tensor).numpy()

        return embedding
    else:
        print(f"No face detected in {image_path}")
        return None

# Function to compare the user image against a small set of celebrity images
def find_celebrity_look_alike(user_image_path, celeb_image_paths, top_n=3):
    user_embedding = extract_face_embedding(user_image_path)

    if user_embedding is None:
        return "No face detected in the user image."

    similarities = []

    # Compare with each celebrity image
    for celeb_path in celeb_image_paths:
        celeb_embedding = extract_face_embedding(celeb_path)
        if celeb_embedding is not None:
            similarity = cosine_similarity(user_embedding, celeb_embedding)[0][0]
            similarities.append((celeb_path, similarity))

    # Sort celebrities by similarity (highest first)
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Return the top N most similar celebrities
    return similarities[:top_n] if similarities else "No face detected in any celebrity images."


In [ ]:
# Step 1: Compute celebrity embeddings and save to a file

celeb_embeddings = {}
for celeb_path in celeba_image_paths:
    celeb_name = os.path.basename(celeb_path)
    embedding = extract_face_embedding(celeb_path)
    if embedding is not None:
        celeb_embeddings[celeb_name] = embedding


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Features shape: torch.Size([1, 100352])
Embeddings shape: torch.Size([1, 128])
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Features shape: torch.Size([1, 100352])
Embeddings shape: torch.Size([1, 128])
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━

In [268]:
import pickle
# Save embeddings to a file (using pickle or numpy)
with open('celeb_embeddings_triplet_loss.pkl', 'wb') as f:
    pickle.dump(celeb_embeddings, f)

print(f"Celebrity embeddings saved successfully!")

Celebrity embeddings saved successfully!
